In this notebook I look at some basic statistics on a graph of user interactions for high scoring questions (score > 75).  The resulting graph is rendered in D3 and allows interactive exploration of the network.

See my blog for a detailed explanation of the approach <a href='https://aster-community.teradata.com/community/learn-aster/aster-works/custom-visualizations-using-d3-and-jupyter'>here</a>.

In [ ]:
import os
import pandas as pd
import networkx as nx
import numpy as np
import json
from IPython.display import Javascript

In [ ]:
answers = pd.read_csv('../input/Answers.csv', encoding='latin-1')
questions = pd.read_csv('../input/Questions.csv', encoding='latin-1')
tags = pd.read_csv('../input/Tags.csv', encoding='latin-1')

In [ ]:
answers = answers.dropna()
questions = questions.dropna()
#tags.dropna()

In [ ]:
answers.OwnerUserId = answers.OwnerUserId.astype(int)
questions.OwnerUserId = questions.OwnerUserId.astype(int)

In [ ]:
questions_sample = questions[questions['Score'] >= 75]
tags_sample = tags[tags['Id'].isin(questions_sample['Id'])]

In [ ]:
result = pd.merge(questions_sample, answers, how = 'inner', left_on = 'Id', right_on = 'ParentId')

In [ ]:
G = nx.from_pandas_dataframe(result, 'OwnerUserId_x', 'OwnerUserId_y')

In [ ]:
self_loops = G.selfloop_edges()
G.remove_edges_from(self_loops)
largest_cc = max(nx.connected_components(G), key=len)
G = G.subgraph(largest_cc)

In [ ]:
print(nx.info(G))

In [ ]:
nx.average_clustering(G)

In [ ]:
nx.density(G)

In [ ]:
centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
tr_clstr = nx.clustering(G)
sq_clstr = nx.square_clustering(G)
eccentricity = nx.eccentricity(G)

In [ ]:
node = []
deg_cent = []
bet_cent = []
tr_cl = []
sq_cl = []
ecc = []
for v in G.nodes():
    node.append(v)
    deg_cent.append(centrality[v])
    bet_cent.append(betweenness_centrality[v])
    tr_cl.append(tr_clstr[v])
    sq_cl.append(sq_clstr[v])
    ecc.append(eccentricity[v])

node_attr = zip(node, deg_cent, bet_cent, tr_cl, sq_cl, ecc)

In [ ]:
edges = G.edges()
edgesJson = json.dumps([{'source': source, 'target': target} for source, target in edges], default = str, indent=2, sort_keys=True)  # called a 'list comprehension'

In [ ]:
nodesJson = json.dumps([{'id': node, 'degree_cent': centrl, 'betweenness_cent': btwn, 'tr_clstr':tr_cl, 'sq_clstr': sq_cl, 'eccentricity': ecc} for node, centrl, btwn, tr_cl, sq_cl, ecc in node_attr], indent=4)

In [ ]:
tagsJson = json.dumps([{'id': id, 'tag': tag} for id, tag in zip(tags_sample['Id'], tags_sample['Tag'])], default = str, indent=4, sort_keys=True)

In [ ]:
controlsJson = '[ { "control": "clear", "abbrev": "CLR", "index": 0 }, { "control": "gravity_up", "abbrev": "H G", "index": 1 }, { "control": "gravity_down", "abbrev": "L G", "index": 2 }, { "control": "degree_cent", "abbrev": "DC", "index": 3 }, { "control": "betweenness_cent", "abbrev": "BC", "index": 4 }, {"control":"tr_clstr", "abbrev": "TC", "index" : 5},{ "control": "sq_clstr", "abbrev": "SQC", "index": 6 }, { "control": "eccentricity", "abbrev": "ECC", "index": 7 } ] '

In [ ]:
Javascript("""
           window.nodes={};
           """.format(nodesJson))

In [ ]:
Javascript("""
           window.edges={};
           """.format(edgesJson))

In [ ]:
Javascript("""
           window.tags={};
           """.format(tagsJson))

In [ ]:
Javascript("""
           window.controls={};
           """.format(controlsJson))

In [ ]:
%%javascript
require.config({
    paths: {
       d3: '//cdnjs.cloudflare.com/ajax/libs/d3/4.8.0/d3.min'
    }
});

In [ ]:
%%javascript
require(['d3'], function(d3){
    // YOUR CUSTOM D3 CODE GOES HERE:
    try{ 
        $("#chart1").remove();
        //create canvas
        element.append("<div id='chart1'></div>");
        $("#chart1").width("850px");
        $("#chart1").height("600px");  
        var margin = {top: 40, right: 10, bottom: 10, left: 10};
        var width = 850 - margin.left - margin.right;
        var height = 600 - margin.top - margin.bottom;
        var forceCenterOffset = {x: 50, y: 50}
        var svg = d3.select("#chart1").append("svg")
            .style("position", "relative")
            .attr("width", width + "px")
            .attr("height", (height) + "px")
            .append("g")
            .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
        var simulation = d3.forceSimulation()
            .force("link", d3.forceLink().id(function(d) { return d.id; }).strength(0.5))
            .force("charge", d3.forceManyBody().strength(-5))
            .force("center", d3.forceCenter(width / 2 - forceCenterOffset.x, height / 2- forceCenterOffset.y));
        var colorScale = d3.scaleLinear().range(["#6b24a5", "#ffffff"]);
        var strokeWidth = 1.0, cSize = 4;
        // CONTROL BOXES:
        var controlBoxes = svg.append("g")
           .attr("class", "control-boxes")
           .attr("transform","translate("+margin.left+","+margin.top+")");;
        // LEGEND & LABELS:
        svg.append("text")
            .style("font","14px sans-serif")
            .attr("transform", "translate(175,0)")
            .text("Click and hold to highlight connections.  Boxes on the right adjust graph settings.")
            .style("fill","#000000");
        var legendSize = {width: 20, height: 200};
        var colorLegendYScale = d3.scaleLinear().range([0, legendSize.height]);
        var div = d3.select("#chart1").append("div")
            .attr("class", "graph-tooltip")
            .style("opacity", 0)
            .style("z-index", 1);
        var colorLegend = svg
            .append("g")
            .attr("class", "legend")
            .attr("transform","translate("+margin.left+","+margin.top+")");
        var linearGradient = colorLegend.append("defs")
            .append("linearGradient")
            .attr("id", "linear-gradient");
        linearGradient
          .attr("x1", "0%")
          .attr("y1", "0%")
          .attr("x2", "0%")
          .attr("y2", "100%");
        linearGradient.selectAll("stop")
          .data(colorScale.range())
          .enter()
          .append("stop")
          .attr("offset", function(d,i) { return i/(colorScale.range().length-1); })
          .attr("stop-color", function(d) { return d; })
          .attr('stop-opacity', 1);
        colorLegend
          .append("rect")
          .attr("width", legendSize.width)
          .attr("height", legendSize.height)
          .attr("transform", "translate(0,0)")
          .style("fill", "url(#linear-gradient)")
          .style("stroke", "black");

        // DRAW THE SIMULATION:
        function drawSimulation(nodes, edges){
            var nodeAttrSelection = "degree_cent";
            setColorScale(nodes, nodeAttrSelection);
            var link = svg.append("g")
              .attr("class", "links")
              .selectAll("line")
              .data(edges)
              .enter()
              .append("line")
              .attr("class", "edge")
              .style("stroke-width", 1.5)
              .style("stroke", "#bbb");
            var node = svg.append("g")
              .attr("class", "nodes")
              .selectAll("circle")
              .data(nodes)
              .enter()
              .append("circle")
              .attr("class", "node")
              .attr("r", cSize)
              .style("stroke", "black")
              .style("stroke-width", strokeWidth)
              .style("fill", function(d){return colorScale(nodeAttrAccessor(d, nodeAttrSelection)); })
              .call(d3.drag()
                  .on("start", dragstarted)
                  .on("drag", dragged)
                  .on("end", dragended));
            nodeTooltip(node, nodeAttrSelection);
            simulation
              .nodes(nodes)
              .on("tick", ticked);
            simulation.force("link")
              .links(edges);
            function ticked() {
            link
                .attr("x1", function(d) { return d.source.x; })
                .attr("y1", function(d) { return d.source.y; })
                .attr("x2", function(d) { return d.target.x; })
                .attr("y2", function(d) { return d.target.y; });
            node
                .attr("cx", function(d) { return d.x; })
                .attr("cy", function(d) { return d.y; });
            }
        };

        // DRAG EVENTS:
        function dragstarted(d) {
          if (!d3.event.active) simulation.alphaTarget(0.3).restart();
          d.fx = d.x;
          d.fy = d.y;
          hideOtherNodes(d);
        }
        function dragged(d) {
          d.fx = d3.event.x;
          d.fy = d3.event.y;
        }
        function dragended(d) {
          if (!d3.event.active) simulation.alphaTarget(0);
          d.fx = null;
          d.fy = null;
          showOtherNodes(d);
        }

        // CAST THE DATA TYPE TO NUMBER:
        function castNodeData(nodeData){
          nodeData.forEach(function(d) {
            d.degree_cent = +d.degree_cent;
            d.betweenness_cent = +d.betweenness_cent;
            d.tr_clstr = +d.tr_clstr;
            d.sq_clstr = +d.sq_clstr;
            d.eccentricity = +d.eccentricity;
          })
        }

        // HIDE/SHOW NODES ON DRAG:
        function hideOtherNodes(d){
          var g_nodes = svg.selectAll(".node");
          var g_edges = svg.selectAll(".edge");
          var shownNodes = [];
          g_edges.filter(function (x) {
              if (d.id != x.target.id && d.id != x.source.id )
              {
                return true;
              } else {
                shownNodes.push(x.target.id);    // push ids for nodes connected to dragged node
                shownNodes.push(x.source.id);
                return false;
              }
            })
            .style("stroke", "#bbb")
            .style("stroke-opacity", 0.1)
            .style();      // fade out everything not connected to dragged node
          g_edges.filter(function(x){ return d.id === x.target.id || d.id === x.source.id; })
            .style("stroke", "#000000");
          g_nodes.filter(function (x) { return (shownNodes.indexOf(x.id) === -1); })
            .style("fill-opacity", 0.1)
            .style("stroke-opacity", 0.1)
            .style("stroke", "#000000")
            .style("stroke-width", strokeWidth);
          g_nodes.filter(function (x) { return (shownNodes.indexOf(x.id) != -1); })
            .style("stroke", "#3039e8")
            .transition()
            .duration(200)
            .style("stroke-width", 2*strokeWidth)
            .style("r", 2*cSize);
        };
        function showOtherNodes(d){
          var g_nodes = svg.selectAll(".node");
          var g_edges = svg.selectAll(".edge");
          g_nodes
            .style("fill-opacity", 1)
            .style("stroke-opacity", 1)
            .transition()
            .delay(200)
            .style("r", cSize);
          g_edges
          .style("stroke-opacity", 0.6);
        };

        // VALUE ACCESSOR:
        function nodeAttrAccessor(d, valueType) {
          if (valueType === "degree_cent") {
            return d.degree_cent;
          } else if ( valueType === "betweenness_cent") {
            return d.betweenness_cent;
          } else if ( valueType === "tr_clstr") {
            return d.tr_clstr;
          } else if (valueType === "sq_clstr") {
            return d.sq_clstr;
          } else if (valueType === "eccentricity") {
            return d.eccentricity;
          }
        }

        // CONTROLS:
        function drawControls(controls, nodes){
          var transitionDuration = 75;
          var controlBoxSize = 30;
          var controlBoxScaleUp = 1.33;
          var controlXOffset = 200;
          var g_box = controlBoxes
            .selectAll("g")
            .data(controls)
            .enter()
            .append("g")
            .attr("transform", function (d,i){
              return "translate("+(width - controlXOffset)+","+(i*(controlBoxSize+ 5))+")"
            })
            .attr("class", "controls");
          g_box
            .append("rect")
            .attr("class", "control")
            .attr("width", controlBoxSize)
            .attr("height", controlBoxSize)
            .style("stroke",  function(d){
              if (d.control === "clear") {
                return "#3039e8";
              } else {
                return "black";
              }
            })
            .style("fill", function(d){
              if (d.control === "clear") {
                return "#ffffff";
              } else if (d.control === "gravity_up" || d.control === "gravity_down") {
                return "#b8b9bc"
              } else {
                return "#b592d2"
              }
             });
          g_box
            .append("text")
            .attr("x", 0.08*controlBoxSize)
            .attr("y", 0.6*controlBoxSize)
            .text(function(d){ return d.abbrev ;})
            .style("pointer-events","none")
            ;
          g_box
            .selectAll("rect")
            .on("click", function(d){
              if (d.control === "clear") {
                resetNodeBorder();
              } else if (d.control === "gravity_up") {
                changeGravity("up");
              } else if (d.control === "gravity_down") {
                changeGravity("down");
              } else {
                setNodeAttribute(d.control, nodes);
              }
            })
            .on("mouseover", function(d, i){
              d3.select(this)
                .transition()
                .duration(transitionDuration)
                .attr("width", controlBoxSize*controlBoxScaleUp)
                .attr("height", controlBoxSize*controlBoxScaleUp)
                .style("stroke-width", 2);
                var index = d.index, additionalOffset = (controlBoxScaleUp-1)*controlBoxSize;
              g_box
                .transition()
                .duration(transitionDuration)
                .attr("transform", function (d,i){
                  if ( i > index) {
                    return "translate("+(width - controlXOffset)+","+(i*(controlBoxSize+5)+additionalOffset)+")"
                  } else {
                    return "translate("+(width - controlXOffset)+","+(i*(controlBoxSize+5))+")"
                  }
                })
                controlTooltip(g_box, index);
            })
            .on("mouseout", function(d){
              d3.select(this)
                .transition()
                .duration(transitionDuration)
                .attr("width", controlBoxSize)
                .attr("height", controlBoxSize)
                .style("stroke-width", 1);
              g_box
                .transition()
                .duration(transitionDuration)
                .attr("transform", function (d,i){
                    return "translate("+(width - controlXOffset)+","+(i*(controlBoxSize+ 5))+")"
                })
            });
        };
        // CONTROL FUNCTIONS:
        function resetNodeBorder(){
          var g_nodes = svg.selectAll(".node");
          var g_edges = svg.selectAll(".edge");
          g_nodes
            .style("stroke", "#000000")
            .style("stroke-width", strokeWidth);
          g_edges
            .style("stroke", "#bbb")
        };

        function changeGravity(direction){
          if (direction ==="down") {
            simulation.force("charge", d3.forceManyBody().strength(-25));
            simulation.alphaTarget(0.3).restart();
            setTimeout(function() { simulation.alphaTarget(0); }, 2500);
          } else if (direction === "up") {
            simulation.force("charge", d3.forceManyBody().strength(-5));
            simulation.alphaTarget(0.3).restart();
            setTimeout(function() { simulation.alphaTarget(0); }, 2500);
          } else {
            simulation
              .force("charge", d3.forceManyBody().strength(-5));
          }
        };

        function setNodeAttribute(attributeType, nodes){
          setColorScale(nodes, attributeType);
          var node = svg.selectAll("circle.node")
            .style("fill", function(d){return colorScale(nodeAttrAccessor(d, attributeType)); });
          nodeTooltip(node, attributeType);
        };

        function setColorScale(nodes, attributeType){
          var nodeAttrMax = d3.max(nodes, function(d){ return nodeAttrAccessor(d, attributeType);});
          //var nodeAttrMin = 0;
          var nodeAttrMin = d3.min(nodes, function(d){ return nodeAttrAccessor(d, attributeType);});  
          var nodeAttrScaleAdj = (nodeAttrMax - nodeAttrMin)
          var nodeAttrExtent = [(nodeAttrMax-nodeAttrScaleAdj*0.25), (nodeAttrMin)];
          colorScale.domain(nodeAttrExtent);
          setLegendScale(nodes, attributeType, nodeAttrExtent)
        };

        function nodeTooltip(node, nodeAttrSelection){
          node
            .on("mouseover", function(d) {
              div.transition()
                  .duration(200)
                  .style("opacity", .9);
              div.html("id:"+d.id+ "<br/>" +nodeAttrSelection+": " + nodeAttrAccessor(d, nodeAttrSelection).toFixed(5))
                 .style("left", (d3.event.pageX) + "px")
                 .style("top", (d3.event.pageY) + "px");
              console.log("x: "+d3.event.pageX+"; y: "+d3.event.pageY);
              })
          .on("mouseout", function(d) {
              div.transition()
                  .duration(500)
                  .style("opacity", 0);
          });
        };

        function controlTooltip(cBox, index){
          var tooltipHTML = "";
          switch(index) {
            case 0:
              tooltipHTML = "Clear Selection";
              break;
            case 1:
              tooltipHTML = "High Gravity";
              break;
            case 2:
              tooltipHTML = "Low Gravity";
              break;
            case 3:
              tooltipHTML = "Show Degree Centrality";
              break;
            case 4:
              tooltipHTML = "Show Betweenness Centrality";
              break;
            case 5:
              tooltipHTML = "Show Triangle Clustering";
              break;
            case 6:
              tooltipHTML = "Show Square Clustering";
              break;
            case 7:
              tooltipHTML = "Show Eccentricity";
              break;
          }
          cBox
            .on("mouseover", function(d) {
              div.transition()
                  .duration(200)
                  .style("opacity", .9);
              div.html(tooltipHTML)
                 .style("left", (d3.event.pageX) + "px")
                 .style("top", (d3.event.pageY) + "px");
              console.log("x: "+d3.event.pageX+"; y: "+d3.event.pageY);
              })
          .on("mouseout", function(d) {
              div.transition()
                  .duration(500)
                  .style("opacity", 0);
          });
        };

        // COLOR LEGEND FUNCTIONS:
        function setLegendScale(data, nodeAttrSelection, colorDomain){
          colorLegendYScale.domain(colorDomain);
          var colorLegendYAxis = d3.axisRight(colorLegendYScale);
          colorLegend
                .selectAll(".y.axis")
                .remove(); 
          colorLegend
                .selectAll(".label")
                .remove();
          colorLegend
                .append("g")
                .attr("class","y axis")
                .attr("transform", "translate(25,0)");
          colorLegend.selectAll(".y.axis")
                .call(colorLegendYAxis)
                .append("text")
                .attr("class", "tick")
                .attr("transform", "rotate(-90)")
                .attr("y", 6)
                .attr("dy", ".71em")
                .style("text-anchor", "end");
          colorLegend
                .append("g")
                .attr("class", "label")
                .attr("transform", "translate(-5,200)")
                .append("text")
                .style("font","14px sans-serif")
                .attr("transform", "rotate(-90)")
                .text(nodeAttrSelection)
                .style("fill","#000000");
        };
        
        function setCSS(){
            d3.select("#chart1")
              .style("font", "10px sans-serif");

            controlBoxes.selectAll("text")
              .style("pointer-events", "none");

            d3.select("div.graph-tooltip")
              .style("position", "relative")
              .style("text-align","center")
              .style("width","180px")
              .style("height","28px")
              .style("padding","2px")
              .style("font","12px sans-serif")
              .style("background","lightsteelblue")
              .style("border","0px")
              .style("border-radius","4px")
              .style("pointer-events","none");
            
        };

        /* ************************************************************** */
        // MAIN:
        /* ************************************************************** */
        // GET THE NETWORK DATA AND CALL DRAW FUNCTION
        var nodesData = window.nodes;
        var edgesData = window.edges;
        var controlsData = window.controls;
        
        setCSS();
        castNodeData(nodesData);
        drawSimulation(nodesData, edgesData);
        drawControls(controlsData, nodesData);
    } catch(err) {
        console.log("Viz Error: ");
        console.log(err);
    }
});